In [1]:
%matplotlib inline
import time
import imp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import unidecode



In [2]:
from problem import get_train_data

X_df, y_array = get_train_data()

In [3]:
X_df.head()

,Legal_ID,Name,Activity_code (APE),Address,Zipcode,City,Headcount,Fiscal_year_end_date,Fiscal_year_duration_in_months,Year
0,582296,COMICOB,NaN,BOULOUPARIS BP 15 BOULOUPARIS,NaN,NaN,NaN,2016-12-31,12.0,2016.0
1,582981,CIANFARANI JEAN-MICHEL,NaN,33 AVENUE H. LAFLEUR - VICTOIRE - B.P. 4031 NO...,NaN,NaN,NaN,2016-12-31,12.0,2016.0
2,588541,OK POULET 5EME SARL,NaN,23 RUE PAUL BERT,98800.0,NOUMEA,1.0,2016-12-31,12.0,2016.0
3,589275,INSTITUT DE LA QUALITE S.A.R.L.,NaN,224 RUE J. IEKAWE - COMPLEXE DE LA BELLE-VIE -...,98800.0,NOUMEA,NaN,2016-12-31,12.0,2016.0
4,592972,EXPERT TRADE,NaN,2 RUE DE L ASTROLABE OUEMO MAGENTA,98800.0,NOUMEA,NaN,2016-12-31,12.0,2016.0


In [4]:
award = pd.read_csv('data/award_notices_RAMP.csv.zip', compression='zip')
award.head(3)

C:\Users\adeik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CallID,Publication_date,End_of_call_date,Departments_of_publication,Departments_of_provision,Call_summary,Call_title,Complete_call_description,Total_amount,CPV_classes,...,ID,awarded,description,incumbent_name,incumbent_address,incumbent_zipcode,incumbent_city,incumbent_country,number_of_received_bids,amount
0,16-119770,2016-08-11 00:00:00,2016-09-20,13,NaN,mission de maitrise d'oeuvre relative aux trav...,MISSION DE MAITRISE D'OEUVRE RELATIVE AUX TRAV...,MISSION DE MAITRISE D'OEUVRE RELATIVE AUX TRAV...,NaN,71000000,...,NaN,Yes,NaN,AI PROJECT,"11, avenue de la Capelette",13010,Marseille,FR,13.0,83200.0
1,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,1,Yes,Kit pour recherche d'anticorps anti-IBR sur sé...,ID-VET,310 rue Louis Pasteur,34790,Grabels,FR,2.0,NaN
2,15-46335,2015-03-27 00:00:00,2015-04-25,85,85,fourniture de réactifs immunosérologiques pour...,NaN,Fourniture de réactifs immunosérologiques pour...,NaN,33696200 33696500,...,2,Yes,Kit pour recherche d'anticorps anti-IBR sur mé...,IDEXX MONTPELLIER SAS,323 RUE DE LA GALERA,34090,MONTPELLIER,FR,2.0,NaN


# Preprocessing Pipeline

# Imputing missing values of X_df

In [5]:
# keeping first 2 numbers of APE
t = time.time()
X_df['Activity_code (APE)'] = X_df['Activity_code (APE)'].fillna(value = '00').astype('str').apply(lambda x: x[:2])

# infer zipcode from city

if(True):
    X_df['Zipcode'] = X_df['Zipcode'].astype('str').apply(lambda x: x[:2])
    city_names = X_df[['Zipcode', 'City']].groupby('Zipcode').City.apply(list).values
    zipcodes = X_df[['Zipcode', 'City']].groupby('Zipcode').City.apply(list).index
    zip_dict = {zipcode:list(set(cities)) for zipcode,cities in zip(zipcodes, city_names)}
    
    #remove nans from city lists
    
    for key in zip_dict.keys():
        for i, city in enumerate(zip_dict[key]):
            if type(city) != type('a string'):
                zip_dict[key].pop(i)
           
    # infer zip by checking lists
    
    def infer_zip(city):
        for key in zip_dict:
            if city in zip_dict[key]:
                return key
    
    X_df.loc[X_df['Zipcode'] == 'na', 'Zipcode'] = X_df.loc[X_df['Zipcode'] == 'na', 'City'].apply(infer_zip)
    X_df.loc[X_df['Zipcode'] == 'na', 'Zipcode'] = '00'
    X_df['Zipcode'] = X_df['Zipcode'].fillna('00')
    
# fill name and lower

X_df['Name'] = X_df['Name'].fillna('nan').str.lower().astype('str').apply(unidecode.unidecode)

# Fill headcounts

if(True):
    headcounts = X_df[['Name', 'Headcount']].groupby('Name').mean().fillna(0).to_dict()['Headcount']
    names = X_df['Name'].values
    heads = np.zeros(len(names))

    for i,name in enumerate(names):
        if type(name) == type('a string'):
            heads[i] = headcounts[name]

    X_df['Headcount'] = heads

# unicode and lower

X_df['City'] = X_df['City'].fillna('nan').str.lower().apply(unidecode.unidecode)

# Drop Address

# X_df no longer has NAs

In [6]:
X_df

,Legal_ID,Name,Activity_code (APE),Address,Zipcode,City,Headcount,Fiscal_year_end_date,Fiscal_year_duration_in_months,Year
0,582296,comicob,00,BOULOUPARIS BP 15 BOULOUPARIS,00,nan,0.0,2016-12-31,12.0,2016.0
1,582981,cianfarani jean-michel,00,33 AVENUE H. LAFLEUR - VICTOIRE - B.P. 4031 NO...,00,nan,0.0,2016-12-31,12.0,2016.0
2,588541,ok poulet 5eme sarl,00,23 RUE PAUL BERT,98,noumea,1.0,2016-12-31,12.0,2016.0
3,589275,institut de la qualite s.a.r.l.,00,224 RUE J. IEKAWE - COMPLEXE DE LA BELLE-VIE -...,98,noumea,0.0,2016-12-31,12.0,2016.0
4,592972,expert trade,00,2 RUE DE L ASTROLABE OUEMO MAGENTA,98,noumea,0.0,2016-12-31,12.0,2016.0
...,...,...,...,...,...,...,...,...,...,...
1495943,987320355,ibt concept,18,56 CHEMIN DE SAINT BERNARD,64,bayonne,4.0,2016-12-31,12.0,2016.0
1495944,995750288,dagard,28,NaN,12,boussac,378.0,2013-12-31,12.0,2013.0
1495945,995750288,dagard,28,NaN,12,boussac,378.0,2014-01-31,12.0,2014.0
1495946,995750288,dagard,28,NaN,12,boussac,378.0,2014-12-31,12.0,2014.0


In [7]:
award = pd.read_csv('data/award_notices_RAMP.csv.zip', compression='zip')

city_list = [unidecode.unidecode(i.lower()) for i in list(set(X_df['City'].values))]

def cedex_remover(s):
    s = s.replace('-', ' ').replace(',', ' ')
    tokens = s.split(' ')
    for k in range(len(tokens)):
        for j in range(1, len(tokens)-k + 1):
            if ' '.join(tokens[k:k+j]) in city_list:
                return ' '.join(tokens[k:k+j])
    return None

award.head(3)

print(award.shape)

# unidecode incumbent_name and lower

award['incumbent_name'] = award['incumbent_name'].str.lower().astype('str').apply(unidecode.unidecode)

# preprocess incumbent_city and fill nans
award['incumbent_city'] = award['incumbent_city'].str.lower().astype('str').apply(unidecode.unidecode).apply(cedex_remover).fillna('nan')


# Remove NA in department of provision and keep first 2
award = award[award['Departments_of_publication'].notna()]
award['Departments_of_publication'] = award['Departments_of_publication'].astype('str').apply(lambda x: x.split(',')[0][:2])


(304098, 28)


In [8]:
# city mean FAN

city_mean_fan = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.5).fillna(0).to_dict()['amount']

city_lower_mean = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.25).fillna(0).to_dict()['amount']
city_higher_mean = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.75).fillna(0).to_dict()['amount']

In [9]:
# zip mean FAN

zip_mean_fan = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.5).fillna(0).to_dict()['amount']

zip_lower_mean = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.25).fillna(0).to_dict()['amount']
zip_higher_mean = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.75).fillna(0).to_dict()['amount']

In [10]:
# APE mean FAN
company_list = [unidecode.unidecode(str(i).lower()) for i in list(set(X_df['Name'].values))]

company_awards = award[award['incumbent_name'].isin(company_list)]

ape_companies = X_df[['Name', 'Activity_code (APE)']].groupby('Name')['Activity_code (APE)'].apply(lambda x: list(x)[0])
ape_companies.index = ape_companies.index.str.lower()
ape_companies = ape_companies.to_dict()

def infer_ape(name):
    return ape_companies[name]
company_awards['APE'] = company_awards['incumbent_name'].apply(infer_ape)

ape_mean_fan = company_awards[['APE','amount']].groupby('APE').quantile(0.5).fillna(0).to_dict()['amount']
ape_lower_mean = company_awards[['APE','amount']].groupby('APE').quantile(0.25).fillna(0).to_dict()['amount']
ape_higher_mean = company_awards[['APE','amount']].groupby('APE').quantile(0.75).fillna(0).to_dict()['amount']

C:\Users\adeik\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [11]:
# Actual FAN revenue

fan_revenue = company_awards[['incumbent_name','amount']].groupby('incumbent_name').sum().fillna(0).to_dict()['amount']


In [12]:
# Insert features into X_df

# Ape FAN
if(True):
    def infer_ape_fan(APE):
        try:
            return ape_mean_fan[APE]
        except:
            return 0

    def infer_low_ape_fan(APE):
        try:
            return ape_lower_mean[APE]
        except:
            return 0

    def infer_high_ape_fan(APE):
        try:
            return ape_higher_mean[APE]
        except:
            return 0



    X_df['APE_fan_0.5'] = X_df['Activity_code (APE)'].apply(infer_ape_fan)
    X_df['APE_fan_0.25'] = X_df['Activity_code (APE)'].apply(infer_low_ape_fan)
    X_df['APE_fan_0.75'] = X_df['Activity_code (APE)'].apply(infer_high_ape_fan)

# City FAN
if(True):

    def infer_city_fan(city):
        try:
            return city_mean_fan[city]
        except:
            return 0

    def infer_low_city_fan(city):
        try:
            return city_lower_mean[city]
        except:
            return 0

    def infer_high_city_fan(city):
        try:
            return city_higher_mean[city]
        except:
            return 0


    X_df['city_fan_0.5'] = X_df['City'].apply(infer_city_fan)
    X_df['city_fan_0.25'] = X_df['City'].apply(infer_low_city_fan)
    X_df['city_fan_0.75'] = X_df['City'].apply(infer_high_city_fan)

# Zip_fan
if(True):
    def infer_zip_fan(zipcode):
        try:
            return zip_mean_fan[zipcode]
        except:
            return 0

    def infer_low_zip_fan(zipcode):
        try:
            return zip_lower_mean[zipcode]
        except:
            return 0

    def infer_high_zip_fan(zipcode):
        try:
            return zip_higher_mean[zipcode]
        except:
            return 0

    X_df['zip_fan_0.5'] = X_df['Zipcode'].apply(infer_zip_fan)
    X_df['zip_fan_0.25'] = X_df['Zipcode'].apply(infer_low_zip_fan)
    X_df['zip_fan_0.75'] = X_df['Zipcode'].apply(infer_high_zip_fan)

# Actual FAN
if(True):
    def infer_fan(name):
        try:
            return fan_revenue[name]
        except:
            return 0

    X_df['FAN'] = X_df['Name'].apply(infer_fan)


In [13]:
time.time() - t

203.21637797355652

In [14]:
X_df

,Legal_ID,Name,Activity_code (APE),Address,Zipcode,City,Headcount,Fiscal_year_end_date,Fiscal_year_duration_in_months,Year,APE_fan_0.5,APE_fan_0.25,APE_fan_0.75,city_fan_0.5,city_fan_0.25,city_fan_0.75,zip_fan_0.5,zip_fan_0.25,zip_fan_0.75,FAN
0,582296,comicob,00,BOULOUPARIS BP 15 BOULOUPARIS,00,nan,0.0,2016-12-31,12.0,2016.0,136986.25,41000.0,415331.34,100923.40,2.744464e+04,333992.330,0.000,0.0000,0.00,0.00
1,582981,cianfarani jean-michel,00,33 AVENUE H. LAFLEUR - VICTOIRE - B.P. 4031 NO...,00,nan,0.0,2016-12-31,12.0,2016.0,136986.25,41000.0,415331.34,100923.40,2.744464e+04,333992.330,0.000,0.0000,0.00,0.00
2,588541,ok poulet 5eme sarl,00,23 RUE PAUL BERT,98,noumea,1.0,2016-12-31,12.0,2016.0,136986.25,41000.0,415331.34,2168640.00,1.349000e+06,9612114.330,276484.715,117650.1925,604011.40,0.00
3,589275,institut de la qualite s.a.r.l.,00,224 RUE J. IEKAWE - COMPLEXE DE LA BELLE-VIE -...,98,noumea,0.0,2016-12-31,12.0,2016.0,136986.25,41000.0,415331.34,2168640.00,1.349000e+06,9612114.330,276484.715,117650.1925,604011.40,0.00
4,592972,expert trade,00,2 RUE DE L ASTROLABE OUEMO MAGENTA,98,noumea,0.0,2016-12-31,12.0,2016.0,136986.25,41000.0,415331.34,2168640.00,1.349000e+06,9612114.330,276484.715,117650.1925,604011.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495943,987320355,ibt concept,18,56 CHEMIN DE SAINT BERNARD,64,bayonne,4.0,2016-12-31,12.0,2016.0,104790.00,35000.0,301810.50,176807.50,7.391037e+04,437562.500,84000.000,19544.0000,255800.00,0.00
1495944,995750288,dagard,28,NaN,12,boussac,378.0,2013-12-31,12.0,2013.0,100273.33,32505.0,261880.00,198780.11,1.218311e+05,304672.675,160590.400,34755.4000,394998.42,3194571.47
1495945,995750288,dagard,28,NaN,12,boussac,378.0,2014-01-31,12.0,2014.0,100273.33,32505.0,261880.00,198780.11,1.218311e+05,304672.675,160590.400,34755.4000,394998.42,3194571.47
1495946,995750288,dagard,28,NaN,12,boussac,378.0,2014-12-31,12.0,2014.0,100273.33,32505.0,261880.00,198780.11,1.218311e+05,304672.675,160590.400,34755.4000,394998.42,3194571.47


In [15]:
to_dummy = ['Activity_code (APE)', 'Zipcode', 'Year']
to_drop = ['Name', 'Address', 'City', 'Fiscal_year_end_date']
X_df = pd.get_dummies(X_df, columns = to_dummy).drop(to_drop, axis = 1)

In [15]:
X_df['y'] = y_array

In [16]:
from binascii import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32


def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]



X_train, X_test = split_train_test_by_id(X_df, 0.2, 'Legal_ID')
y_train = X_train['y']
X_train = X_train.drop(['Legal_ID', 'y'], axis = 1)

y_test = X_test['y']
X_test = X_test.drop(['Legal_ID', 'y'], axis = 1)

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

In [20]:
from scipy import sparse
X_train = X_train.to_numpy()
X_train = sparse.csr_matrix(X_train)

poly = PolynomialFeatures(degree = 2)

X_train = poly.fit_transform(X_train)

X_test = X_test.to_numpy()
X_test = sparse.csr_matrix(X_test)
X_test = poly.fit_transform(X_test)

In [21]:
lasso = Lasso(alpha = 1, precompute = True)
lasso.fit(X_train, y_train)

C:\Users\adeik\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.97512943774025e+22, tolerance: 9.981632609498178e+18
  tol, rng, random, positive)


Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=True, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [ ]:
model = SVR(kernel = 'poly', degree = 2)
model.fit(X_train,y_train)

C:\Users\adeik\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [41]:
f
model.score(X_test,y_test)

0.005718304851546785

In [43]:
y_test

2           10481715.0
16          92267790.0
19         145009289.0
24         349524711.0
30          40781049.0
              ...     
1495914      1105975.0
1495915      1055074.0
1495916      1276956.0
1495917       902759.0
1495918      1361306.0
Name: y, Length: 299947, dtype: float64

In [41]:
X_train.shape

(1196001, 200)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


model = Sequential()
model.add(Dense(64, input_shape = X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(1))

optimizer = Adam(lr = 0.1)

model.compile(loss = 'mse', optimizer = optimizer)



In [57]:
model.fit(X_train, y_train.to_numpy(), epochs =5, batch_size = 32)

Train on 1196001 samples
Epoch 1/5
 257248/1196001 [=====>........................] - ETA: 5:46 - loss: 40745120993672320.0000

KeyboardInterrupt: 

In [22]:
y_pred = lasso.predict(X_test)

In [45]:
y_pred

array([99016500.5635376 , 99031360.80163574, 99027625.91430664, ...,
        1137730.16552734,  1137730.16552734,  1200846.23931885])

In [23]:
from sklearn.metrics import mean_squared_error

In [27]:
np.log(np.abs(y_pred - y_test))

2          18.299774
16         15.738368
19         17.642080
24         19.338703
30         17.881786
             ...    
1495914    12.577428
1495915    12.724495
1495916     9.487221
1495917    12.796652
1495918    10.468785
Name: y, Length: 299947, dtype: float64

In [25]:
y_pred

array([99093376.88196306, 99108263.48635395, 99104502.24160202, ...,
        1263765.91166013,  1263765.91166013,  1326106.56396625])

In [32]:
y_pred[0]

99093376.88196306

In [34]:
y_test.to_numpy()[0]

10481715.0

In [40]:
i = 5
y_pred[i] - y_test.to_numpy()[i]

60670786.25974746

In [28]:
y_pred - y_test

2          8.861166e+07
16         6.840473e+06
19        -4.590479e+07
24        -2.504332e+08
30         5.833927e+07
               ...     
1495914    2.899397e+05
1495915    3.358751e+05
1495916   -1.319009e+04
1495917    3.610069e+05
1495918   -3.519944e+04
Name: y, Length: 299947, dtype: float64

# Pyspark


In [16]:
from binascii import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32


def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]



X_train, X_test = split_train_test_by_id(X_df, 0.2, 'Legal_ID')
#y_train = X_train['y']
X_train = X_train.drop(['Legal_ID'], axis = 1)

#y_test = X_test['y']
X_test = X_test.drop(['Legal_ID'], axis = 1)

In [ ]:
import pyspark as spark
from pyspark.sql import SQLContext

sc = spark.SparkContext('local[*]')
sqlcontext = SQLContext(sc)
X_train = sqlcontext.createDataFrame(data = X_train)

X_test = sqlcontext.createDataFrame(data = X_test)

y_train = sqlcontext.createDataFrame(data = y_train)
y_test = sqlcontext.createDataFrame(data = y_train)

In [ ]:
X_test = sqlcontext.createDataFrame(data = X_test)

y_train = sqlcontext.createDataFrame(data = y_train)
y_test = sqlcontext.createDataFrame(data = y_train)

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)